### Analysis code snippets for preprocessed h5 files

Organized as follows:
1. Select runs and filter based on number of hits, including choosing ions, electron and/or photon data
2. Filter based on pulse number
3. Calibrate runs for m/q values
4. Heatmap and electron time of flight plots
5. Fish plots
6. Intensity dependent plots including into time of flight plots, waterfall plots and heatmaps
7. Presentation plots including fish plot, heatmaps, electron and ion data
8. Covariances between ion data and between ion and electron data
9. PNCCD photon data

In [ ]:
Need to install reading methods the first time:

pip install --user tables   ### to read dataframe
pip install h5netcdf        ### to read xarrays

# Imports and functions

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter

In [ ]:
TIME_BETWEEN_PULSES = 3.54462e-6
CHANNELS_PER_PULSE = 14080
channel_time = TIME_BETWEEN_PULSES/CHANNELS_PER_PULSE


    
def read(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, etof, pnccd



def read_ions_electrons(runid,tof_limit=None):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof in that order'
    'tof_limit limiting electron time of flight loaded in'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    etof = xr.open_dataarray(filename, group="etof")

    if type(tof_limit) == int:
        max_coord = int(tof_limit/channel_time)
        etof = etof[:max_coord]
    
    return dfevent, dfpulse, etof



def read_ions_photons(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return dfevent, dfpulse, pnccd



def read_ion(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event and per pulse'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    
    dfevent = pd.read_hdf(filename, 'dfevent')
    dfpulse = pd.read_hdf(filename, 'dfpulse')
    
    return dfevent, dfpulse



def read_pnccd(runid):
    'Read the preprocessed data of run with ID runid saved in the h5 file with a corresponding name'
    'Outputs dataframes per event, per pulse, and xarrays etof, pnccd in that order'
    
    filename = '../preprocess/datarun' + str(runid) + '.h5'
    pnccd = xr.open_dataarray(filename, group="pnccd")
    
    return pnccd



def events_selection(runs,thresholds,num_pulses=None):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_etofs1 = list()
    selected_pnccds1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_etofs2 = list()
        selected_pnccds2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_etofs3 = list()
        selected_pnccds3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        dfevent, dfpulse, etof, pnccd = read(run)
        
        selections = list()
        
        plt.figure()
        plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,c='black',label='All pulses')
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_etof1 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse1.pulseId))
        selected_pnccd1 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse1.trainId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_etof1, selected_pnccd1))
        plt.scatter(selected_dfpulse1.pulseId,selected_dfpulse1.nevents_pulse,c='r',label=f'Between {lower_threshold1} and {upper_threshold1}')
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_etof2 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse2.pulseId))
            selected_pnccd2 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse2.trainId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_etof2, selected_pnccd2))
            plt.scatter(selected_dfpulse2.pulseId,selected_dfpulse2.nevents_pulse,c='blue',label=f'Between {lower_threshold2} and {upper_threshold2}')
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_etof3 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse3.pulseId))
            selected_pnccd3 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse3.trainId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_etof3, selected_pnccd3))
            plt.scatter(selected_dfpulse3.pulseId,selected_dfpulse3.nevents_pulse,c='g',label=f'Between {lower_threshold3} and {upper_threshold3}')  
        
        dataframes[run] = selections
          
        plt.xlabel('Pulse ID')
        plt.ylabel('Number of events per pulse')
        plt.legend()
        plt.title(f'Events per pulse with respect to pulse ID for run {run}')
        plt.show()

        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_etofs1.append(values[0][2])
        selected_pnccds1.append(values[0][3])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_etofs2.append(values[1][2])
            selected_pnccds2.append(values[1][3])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_etofs3.append(values[2][2])
            selected_pnccds3.append(values[2][3])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_etof1 = xr.concat(selected_etofs1, dim='pulseId')
    merged_pnccd1 = xr.concat(selected_pnccds1, dim='pulseId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_etof1, merged_pnccd1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_etof2 = xr.concat(selected_etofs2, dim='pulseId')
        merged_pnccd2 = xr.concat(selected_pnccds2, dim='pulseId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_etof2, merged_pnccd2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_etof3 = xr.concat(selected_etofs3, dim='pulseId')
        merged_pnccd3 = xr.concat(selected_pnccds3, dim='pulseId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_etof3, merged_pnccd3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    
    return merged_selection



def events_selection_ions_electrons(runs,thresholds,num_pulses=None,tof_limit=None):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_etofs1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_etofs2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_etofs3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        dfevent, dfpulse, etof = read_ions_electrons(run,tof_limit)
        
        selections = list()
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_etof1 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse1.pulseId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_etof1))
        plt.scatter(selected_dfpulse1.pulseId,selected_dfpulse1.nevents_pulse,c='r',label=f'Between {lower_threshold1} and {upper_threshold1}')
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_etof2 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse2.pulseId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_etof2))
            plt.scatter(selected_dfpulse2.pulseId,selected_dfpulse2.nevents_pulse,c='blue',label=f'Between {lower_threshold2} and {upper_threshold2}')
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_etof3 = etof.sel(pulseId=etof.coords['pulseId'].isin(selected_dfpulse3.pulseId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_etof3))
            plt.scatter(selected_dfpulse3.pulseId,selected_dfpulse3.nevents_pulse,c='g',label=f'Between {lower_threshold3} and {upper_threshold3}')  
        
        dataframes[run] = selections
        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_etofs1.append(values[0][2])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_etofs2.append(values[1][2])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_etofs3.append(values[2][2])
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_etof1 = xr.concat(selected_etofs1, dim='pulseId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_etof1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
           
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_etof2 = xr.concat(selected_etofs2, dim='pulseId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_etof2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}") 
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_etof3 = xr.concat(selected_etofs3, dim='pulseId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_etof3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    return merged_selection



def events_selection_ions_photons(runs,thresholds,num_pulses=None):
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    selected_pnccds1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
        selected_pnccds2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
        selected_pnccds3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        print('Processing run number',run)
        
        dfevent, dfpulse, pnccd = read_ions_photons(run)
        
        selections = list()
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selected_pnccd1 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse1.trainId))
        selections.append((selected_dfevent1, selected_dfpulse1, selected_pnccd1))
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selected_pnccd2 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse2.trainId))
            selections.append((selected_dfevent2, selected_dfpulse2, selected_pnccd2))
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selected_pnccd3 = pnccd.sel(trainId=pnccd.coords['trainId'].isin(selected_dfpulse3.trainId))
            selections.append((selected_dfevent3, selected_dfpulse3, selected_pnccd3))
            
        dataframes[run] = selections
        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        selected_pnccds1.append(values[0][2])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            selected_pnccds2.append(values[1][2])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
            selected_pnccds3.append(values[2][2])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_pnccd1 = xr.concat(selected_pnccds1, dim='trainId')
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1, merged_pnccd1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
    
        merged_pnccd2 = xr.concat(selected_pnccds2, dim='trainId')
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2, merged_pnccd2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
    
        merged_pnccd3 = xr.concat(selected_pnccds3, dim='trainId')
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3, merged_pnccd3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")

    
    return merged_selection



def ion_selection(runs,thresholds,num_pulses=None):
    'Only handles ion data'
    'Reads one or multiple runs from h5 files'
    'Makes a pulse selection based on the number of events per pulse between the defined thresholds'
    'If multiple runs are passed, will merge the runs, once hit selected'
    'Thresholds can be between one and three tuples (lower threshold, upper threshold)'
    'Downsamples by num_pulses'
    
    lower_threshold1, upper_threshold1 = thresholds[0]
    selected_dfevents1 = list()
    selected_dfpulses1 = list()
    
    if len(thresholds) > 1:
        lower_threshold2, upper_threshold2 = thresholds[1]
        selected_dfevents2 = list()
        selected_dfpulses2 = list()
    
    if len(thresholds) > 2:
        lower_threshold3, upper_threshold3 = thresholds[2]  
        selected_dfevents3 = list()
        selected_dfpulses3 = list()
    
    dataframes = dict()
    
    if type(num_pulses) == int:
        num_pulses_run = int(num_pulses/len(runs))
    
    for run in runs:
        
        print('Handling run', run)
        dfevent, dfpulse = read_ion(run)
        
        selections = list()
        # plt.figure()
        # plt.scatter(dfpulse.pulseId,dfpulse.nevents_pulse,c='black',label='All pulses')
        
        if type(num_pulses) == int:
            dfpulse = dfpulse.sample(n=num_pulses_run)
            
        selected_dfpulse1 = dfpulse[lower_threshold1 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold1]
        selected_dfevent1 = dfevent[dfevent.pulseId.isin(selected_dfpulse1.pulseId)]
        selections.append((selected_dfevent1, selected_dfpulse1))
        
        if len(thresholds) > 1:
            
            selected_dfpulse2 = dfpulse[lower_threshold2 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold2]
            selected_dfevent2 = dfevent[dfevent.pulseId.isin(selected_dfpulse2.pulseId)]
            selections.append((selected_dfevent2, selected_dfpulse2))
            
        if len(thresholds) > 2:
            
            selected_dfpulse3 = dfpulse[lower_threshold3 < dfpulse.nevents_pulse][dfpulse.nevents_pulse < upper_threshold3]
            selected_dfevent3 = dfevent[dfevent.pulseId.isin(selected_dfpulse3.pulseId)]
            selections.append((selected_dfevent3, selected_dfpulse3))
        
        dataframes[run] = selections
          
        # plt.xlabel('Pulse ID')
        # plt.ylabel('Number of events per pulse')
        # plt.legend()
        # plt.title(f'Events per pulse with respect to pulse ID for run {run}')
        # plt.show()

        
    for key, values in dataframes.items():
        
        selected_dfevents1.append(values[0][0])
        selected_dfpulses1.append(values[0][1])
        
        if len(thresholds) > 1:
            selected_dfevents2.append(values[1][0])
            selected_dfpulses2.append(values[1][1])
            
        if len(thresholds) > 2: 
            selected_dfevents3.append(values[2][0])
            selected_dfpulses3.append(values[2][1])
        
        
    merged_selection = list()
    
    merged_dfevent1 = pd.concat(selected_dfevents1)
    merged_dfevent1.reset_index(drop=True, inplace=True)
    
    merged_dfpulse1 = pd.concat(selected_dfpulses1)
    merged_dfpulse1.reset_index(drop=True, inplace=True)
    
    merged_selection.append((merged_dfevent1, merged_dfpulse1))
    
    print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold1} and {upper_threshold1} events: {len(merged_dfpulse1)}")
       
        
    if len(thresholds) > 1:
        merged_dfevent2 = pd.concat(selected_dfevents2)
        merged_dfevent2.reset_index(drop=True, inplace=True)
    
        merged_dfpulse2 = pd.concat(selected_dfpulses2)
        merged_dfpulse2.reset_index(drop=True, inplace=True)
        
        merged_selection.append((merged_dfevent2, merged_dfpulse2))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold2} and {upper_threshold2} events: {len(merged_dfpulse2)}")
    
    
    if len(thresholds) > 2:
        merged_dfevent3 = pd.concat(selected_dfevents3)
        merged_dfevent3.reset_index(drop=True, inplace=True)
    
        merged_dfpulse3 = pd.concat(selected_dfpulses3)
        merged_dfpulse3.reset_index(drop=True, inplace=True)
        
        merged_selection.append((merged_dfevent3, merged_dfpulse3))
        
        print(f"Number of pulses selected across {len(runs)} run(s) between {lower_threshold3} and {upper_threshold3} events: {len(merged_dfpulse3)}")
        
    
    return merged_selection



